# Read the predictions from the annot xml files and convert them into GIS shapefiles

In [1]:
# cell 1
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# cell 2
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 14.2 MB/s 
     |████████████████████████████████| 16.7 MB 52.5 MB/s 
     |████████████████████████████████| 6.3 MB 39.8 MB/s 


In [6]:
import os
#batch_group = "pa_northcentral_2019_b19"
#batch_group = "pa_allentown_2016"
#batch_group = "pa_3_county_south_central_2018_d18"
#batch_group = "pa_luzernecounty_2018"
#batch_group = "pa_sandy_2014"
#batch_group = "pa_south_central_b1_2017"
#batch_group = "pa_south_central_b2_2017"
#batch_group = "pa_dauphin_2016"
#batch_group = "pa_westernpa_2019_d20"
#batch_group = "de_delawarevalley_hd_2015"
batch_group ="md-pa_sandysupp_2014"

dem_tif_fp = os.path.join("/content/drive/MyDrive/crane_pennsylvania/dem/",batch_group)
slope_tif_fp = os.path.join("/content/drive/MyDrive/crane_pennsylvania/slope/",batch_group)
slope_tif_tiles_fp = os.path.join(slope_tif_fp,"tiles640/")
slope_tif_tiles_jpgs_fp = os.path.join(slope_tif_tiles_fp,"jpgs/")
prediction_xmls_fp = '/content/drive/MyDrive/crane_pennsylvania/predictions/project_'+batch_group+'/xmls/'
prediction_fp = '/content/drive/MyDrive/crane_pennsylvania/predictions/project_'+batch_group+'/'
#dem_tif_fp, slope_tif_fp, slope_tif_tiles_fp, slope_tif_tiles_jpgs_fp, prediction_xmls_fp


training_fp = '/content/drive/MyDrive/crane_pennsylvania/training_images/'
training_tifs_fp = os.path.join(training_fp, 'tifs/')
training_jpgs_fp = os.path.join(training_fp, 'jpgs/')
training_xmls_fp = os.path.join(training_fp, 'xmls/')
training_txts_fp = os.path.join(training_fp, 'txts/')

training_yolov5pytorch_fp = os.path.join(training_fp, 'yolov5pytorch/')

#test
yolo_test_fp = os.path.join(training_yolov5pytorch_fp, 'test/')
yolo_test_images_fp = os.path.join(yolo_test_fp, 'images/')

#set targets to collect preductions from
target_xmls_fp = prediction_xmls_fp
target_tifs_fp = slope_tif_tiles_fp
target_jpgs_fp = slope_tif_tiles_jpgs_fp

In [7]:
import os
def check_number_of_files(folder_path, file_extension):
    counter = 0
    for filename in os.listdir(folder_path):
        ext = filename[-1*(len(file_extension)):]
        if(ext == file_extension):
            counter = counter + 1            
    print("found: ",counter," files in ", folder_path)

print("These totals should be equal:")
check_number_of_files(target_xmls_fp, ".xml")



These totals should be equal:
found:  17920  files in  /content/drive/MyDrive/crane_pennsylvania/predictions/project_md-pa_sandysupp_2014/xmls/


In [9]:
check_number_of_files(target_tifs_fp, ".tif")

found:  17920  files in  /content/drive/MyDrive/crane_pennsylvania/slope/md-pa_sandysupp_2014/tiles640/


In [10]:
check_number_of_files(target_jpgs_fp, ".jpg")

found:  17920  files in  /content/drive/MyDrive/crane_pennsylvania/slope/md-pa_sandysupp_2014/tiles640/jpgs/


In [11]:
# Dictionary of slope_tifs
# Sometimes there are input/output errors and timeouts with Google Drive.  If odd results like a wrong count are produced, re-run this

#dem_tif_fp, slope_tif_fp, slope_tif_tiles_fp, slope_tif_tiles_jpgs_fp

slope_tifs_dict = {}
for file_name in os.listdir(target_tifs_fp):
    origin_file_path = os.path.join(slope_tif_tiles_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".tif"):
            # be careful comparing a partial filename
            slope_tifs_dict[file_name[:-4]]=file_name

# Dictionary of tifs
slope_jpgs_dict = {}
for file_name in os.listdir(target_jpgs_fp):
    origin_file_path = os.path.join(slope_tif_tiles_jpgs_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".jpg"):
            # be careful comparing a partial filename
            slope_jpgs_dict[file_name[:-4]]=file_name
#print(tifs_dict)

print("There are: ",len(slope_tifs_dict)," tif files in ", target_tifs_fp)
print("There are: ",len(slope_jpgs_dict)," jpg files in ", target_jpgs_fp)
for dict_key in slope_tifs_dict:
    if not dict_key in slope_jpgs_dict:
        print(dict_key, " (tif) is not in jpgs directory.")

for dict_key in slope_jpgs_dict:
    if not dict_key in slope_tifs_dict:
        print(dict_key, " (jpg) is not in tifs directory.")

There are:  17920  tif files in  /content/drive/MyDrive/crane_pennsylvania/slope/md-pa_sandysupp_2014/tiles640/
There are:  17920  jpg files in  /content/drive/MyDrive/crane_pennsylvania/slope/md-pa_sandysupp_2014/tiles640/jpgs/


In [12]:
slope_jpgs_dict = {}
for file_name in os.listdir(target_jpgs_fp):
    origin_file_path = os.path.join(target_jpgs_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".jpg"):
            # be careful comparing a partial filename
            slope_jpgs_dict[file_name[:-4]]=file_name

In [13]:
xmls_dict = {}
for file_name in os.listdir(target_xmls_fp):
    origin_file_path = os.path.join(target_xmls_fp ,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".xml"):
            # be careful comparing a partial filename
            xmls_dict[file_name[:-4]]=file_name

print("There are: ",len(xmls_dict)," xml files in ", target_xmls_fp)
for dict_key in xmls_dict:
    if not dict_key in slope_jpgs_dict:
        print(dict_key, " (xml) is not in jpgs directory.")

#print("There are: ",len(slope_jpgs_dict)," jpg files in ", slope_jpgs_fp)
for dict_key in slope_jpgs_dict:
    if not dict_key in xmls_dict:
        print(dict_key, " (jpg) is not in xmls directory.")

There are:  17920  xml files in  /content/drive/MyDrive/crane_pennsylvania/predictions/project_md-pa_sandysupp_2014/xmls/


In [14]:
# cell 3 - Run this load these functions
def get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,x,y):
    # supposing x and y are your pixel coordinate this 
    # is how to get the coordinate in space.
    posX = px_w * x + rot1 * y + xoffset
    posY = rot2 * x + px_h * y + yoffset

    # shift to the center of the pixel
    posX += px_w / 2.0
    posY += px_h / 2.0
    return posX,posY

# rewite this to pass an open dataset

def get_poly_from_geotif_with_x_y(geotif_fp,minx,miny,maxx,maxy):
    ds = gdal.Open(geotif_fp)
    # open the dataset and get the geo transform matrix

    xoffset, px_w, rot1, yoffset, rot2,px_h = ds.GetGeoTransform()

    #print("xoffset, px_w, rot1, yoffset, px_h, rot2",xoffset, px_w, rot1, yoffset, px_h, rot2)
    #print("minx,miny,maxx,maxy",minx,miny,maxx,maxy)

    pos1x,pos1y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,minx,miny)
    pos2x,pos2y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,minx,maxy)
    pos3x,pos3y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,maxx,maxy)
    pos4x,pos4y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,maxx,miny)
    coords = [(pos1x,pos1y), (pos2x,pos2y), (pos3x,pos3y), (pos4x,pos4y)]

    #print("pos",pos1x,pos1y,pos2x,pos2y,pos3x,pos3y,pos4x,pos4y)
    poly = Polygon(coords)
    
    return poly

def get_poly_from_ds_with_x_y(ds,minx,miny,maxx,maxy):
    
    # open the dataset and get the geo transform matrix

    xoffset, px_w, rot1, yoffset, rot2,px_h = ds.GetGeoTransform()

    #print("xoffset, px_w, rot1, yoffset, px_h, rot2",xoffset, px_w, rot1, yoffset, px_h, rot2)
    #print("minx,miny,maxx,maxy",minx,miny,maxx,maxy)

    pos1x,pos1y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,minx,miny)
    pos2x,pos2y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,minx,maxy)
    pos3x,pos3y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,maxx,maxy)
    pos4x,pos4y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,maxx,miny)
    coords = [(pos1x,pos1y), (pos2x,pos2y), (pos3x,pos3y), (pos4x,pos4y)]

    #print("pos",pos1x,pos1y,pos2x,pos2y,pos3x,pos3y,pos4x,pos4y)
    poly = Polygon(coords)
    
    return poly 


In [ ]:
"""
import os
#batch_group = "pa_sandy_2014/"
batch_group = "pa_south_central_b1_2017"
dem_tif_fp = os.path.join("/content/drive/MyDrive/crane_pennsylvania/dem/",batch_group)
slope_tif_fp = os.path.join("/content/drive/MyDrive/crane_pennsylvania/slope/",batch_group)
slope_tif_tiles_fp = os.path.join(slope_tif_fp,"tiles/")
slope_tif_tiles_jpgs_fp = os.path.join(slope_tif_tiles_fp,"jpgs/")
prediction_xmls_fp = '/content/drive/MyDrive/crane_pennsylvania/predictions/project_'+batch_group+'/xmls/'
prediction_fp = '/content/drive/MyDrive/crane_pennsylvania/predictions/project_'+batch_group+'/'
#dem_tif_fp, slope_tif_fp, slope_tif_tiles_fp, slope_tif_tiles_jpgs_fp, prediction_xmls_fp
"""

In [15]:
# cell 4 - Run this to load a dictionary of files to process
# Looping through them repeatedly takes a long time.
# Instead, create a dictionary of files indexed by area. Each entry holds a list of matching files
# This makes it easier to process these files by area.


import csv
import os
from os import listdir
construction_type = "charcoal_hearth_hill"
#cfg_name = 'cfg20200826T2315'

#make a dict of all the areas + pan (or pas)
area_crs_dict = {}
area_crs_dict[batch_group] = []

# Now that the dictionary is created, add all of the matching files as a list linked to the entry.
# This dictionary will be used below.

annot_prediction_folder = prediction_xmls_fp

for annot_filename in listdir(annot_prediction_folder):
    annot_area = batch_group
    #print(annot_filename, annot_area)
    area_node = area_crs_dict[annot_area]
    area_node.append(annot_filename)
print(area_crs_dict) 

{'md-pa_sandysupp_2014': ['USGS_one_meter_x42y436_MD_PA_SandySupp_2014r09c12.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r09c13.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r09c14.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r09c15.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c00.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c01.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c02.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c03.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c04.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c05.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c06.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c07.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c08.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c09.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c10.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c11.xml', 'USGS_one_meter_x42y436_MD_PA_SandySupp_2014r10c12.xml', 'USGS

In [16]:
# cell 5
construction_type = "charcoal_hearth_hill"
cfg_name = 'cfg20200826T2315'
model_epoch='0016'

split_tifs_folder = slope_tif_tiles_fp
# display image with masks and bounding boxes

from xml.etree import ElementTree
#https://gis.stackexchange.com/questions/92207/split-a-large-geotiff-into-smaller-regions-with-python-and-gdal
import os
from os import listdir
import numpy
from osgeo import gdal, osr
import math
from itertools import chain
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import gdalnumeric

import cv2
def put_preds_in_shp(state_area,state_area_num_crs):

    pred_polys = gpd.GeoDataFrame()
    pred_polys['geometry'] = None
    
    pred_polys.crs = ("EPSG:" + str(state_area_num_crs))    
    #pred_polys.geometry = pred_polys.geometry.crs(epsg=state_area_num_crs)
    pred_polys.geometry.crs = ("EPSG:" + str(state_area_num_crs))
    print("pred_polys.crs",pred_polys.crs, pred_polys.geometry.crs)    

    #Store the results in XML    
    class_names = construction_type

    # find all images
    pa = area_crs_dict[str(state_area)]
    files_to_process = len(pa)
    print("files to process: ",files_to_process)
    file_counter = 0
    for annot_filename in pa:
        file_counter = file_counter + 1
        #print(annot_filename)
        #process only the files for this state land area, since other areas may not match crs
        #if annot_filename.startswith(state_area_num):
        tree = ElementTree.parse(annot_prediction_folder+annot_filename)
        print(file_counter, " of ", files_to_process , annot_prediction_folder+annot_filename)
        
        #print(tree)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
    
        fn_image = root.find('./filename').text
        #object_present = root.find('./object_present').text
        fn_base = fn_image[:-4]
        #print(fn_base)
        box_num=0
        if len(root.findall('./object'))>0:
            tif_file_path = os.path.join(split_tifs_folder+(fn_base+".tif"))
            ds = gdal.Open(tif_file_path)
            print(tif_file_path)
        for obj in root.findall('./object'):
            score = obj.find('score').text
            if float(score)>.7:
                box = obj.find('bndbox')
                box_num=obj.find('number').text
                box_num_pad = "00"+str(box_num)
                box_num_pad = box_num_pad[-2:]
                xmin = int(box.find('xmin').text)
                ymin = int(box.find('ymin').text)
                xmax = int(box.find('xmax').text)
                ymax = int(box.find('ymax').text)
                if(ymin>ymax):
                    ytemp = ymin
                    ymin = ymax
                    ymax=ytemp
                if(xmin>xmax):
                    xtemp = xmin
                    xmin = xmax
                    xmax=xtemp            
                coors = [xmin, ymin, xmax, ymax]
                #print("score", score, coors)            
                try:
                    #changed!
                    pred_poly = get_poly_from_ds_with_x_y(ds,xmin,ymin,xmax,ymax)
                    new_pp_row = {'id':fn_base+box_num_pad, 'geometry':pred_poly, 'score':score}
                    pred_polys = pred_polys.append(new_pp_row, ignore_index=True)
                except:
                    print("ERROR with file above ^")
            #print("pred_polys.crs",pred_polys.crs, pred_polys.geometry.crs)
            pred_polys.geometry.crs = ("EPSG:" + str(state_area_num_crs))
            #print("pred_polys.crs",pred_polys.crs, pred_polys.geometry.crs)
        

    outfolder = prediction_fp
    if not os.path.exists(outfolder):
        os.makedirs(outfolder)
    outfp = os.path.join(outfolder,(state_area + "_predictions.shp"))
                         
# Write the data into that Shapefile
    if not pred_polys.empty:
        #pred_polys.head()
        #pred_polys = pred_polys.to_crs({'init':'epsg:4326'})
        #pred_polys = pred_polys.to_crs(epsg = 26918)
        print("pred_polys.crs",pred_polys.crs, pred_polys.geometry.crs)
        pred_polys.to_file(outfp)
        print("File written to", outfp)

        """
        crs_4326 = 4326
        pred_polys.geometry = pred_polys.geometry.to_crs(crs=crs_4326)
        pred_polys.to_crs(crs=crs_4326)
        pred_polys = pred_polys.to_crs(epsg=crs_4326)
        
        #pred_polys = pred_polys.set_crs(epsg = 4326)
        #pred_polys.head()
        outfp = os.path.join(outfolder,("4326_" + state_area + "_predictions.shp"))
        # Write the data into that Shapefile
        pred_polys.to_file(outfp)
        """
 

In [17]:
# cell 6
put_preds_in_shp(batch_group,26918)

Streaming output truncated to the last 5000 lines.
13333  of  17920 /content/drive/MyDrive/crane_pennsylvania/predictions/project_md-pa_sandysupp_2014/xmls/USGS_one_meter_x40y435_MD_PA_SandySupp_2014r09c03.xml
13334  of  17920 /content/drive/MyDrive/crane_pennsylvania/predictions/project_md-pa_sandysupp_2014/xmls/USGS_one_meter_x40y435_MD_PA_SandySupp_2014r09c04.xml
13335  of  17920 /content/drive/MyDrive/crane_pennsylvania/predictions/project_md-pa_sandysupp_2014/xmls/USGS_one_meter_x40y435_MD_PA_SandySupp_2014r09c05.xml
13336  of  17920 /content/drive/MyDrive/crane_pennsylvania/predictions/project_md-pa_sandysupp_2014/xmls/USGS_one_meter_x40y435_MD_PA_SandySupp_2014r09c06.xml
13337  of  17920 /content/drive/MyDrive/crane_pennsylvania/predictions/project_md-pa_sandysupp_2014/xmls/USGS_one_meter_x40y435_MD_PA_SandySupp_2014r09c07.xml
13338  of  17920 /content/drive/MyDrive/crane_pennsylvania/predictions/project_md-pa_sandysupp_2014/xmls/USGS_one_meter_x40y435_MD_PA_SandySupp_2014r09c08

In [ ]:
# make a layer of just polygons inside forested areas

import os
import geopandas as gpd

predictions_path = "/content/drive/MyDrive/crane_pennsylvania/predictions/project_" + batch_group + "/" + batch_group + "_predictions.shp"
forest_polygons_path = "/content/drive/MyDrive/crane_pennsylvania/predictions/project_" + batch_group + "/" + batch_group + "_forest_polygons.shp"
predictions_in_forest_path = "/content/drive/MyDrive/crane_pennsylvania/predictions/project_" + batch_group + "/" + batch_group + "_predictions_in_forest.shp"

# Create an empty geopandas GeoDataFrame for predictions in a forested area
predictions_in_forest_df = gpd.GeoDataFrame()
predictions_in_forest_df['geometry'] = None

predictions_in_forest_df.crs = ('EPSG:26918')

if os.path.exists(predictions_path) and os.path.exists(forest_polygons_path):
    predictions_polys = gpd.read_file(predictions_path)
    print("Total raw predictions:", len(predictions_polys))
    forest_polys = gpd.read_file(forest_polygons_path)
    print("crs", predictions_polys.crs,forest_polys.crs )
    print(type(forest_polys))
    print(list(forest_polys))
    print("lidar_tiles_polys",predictions_polys.shape, "land_use_polys",forest_polys.shape)
    forest_geoseries = forest_polys['geometry']
    print(type(forest_geoseries))

    area_crs = 26918
    matched_tiles_polys = list()    
    predictions_polys.to_crs(area_crs)
    forest_polys.to_crs(area_crs)
    row_counter = 0 
    for index, row in predictions_polys.iterrows():
       #print("row",row)

       pred_poly = row[2]
       #print("-----")
       #print("pred_poly",type(pred_poly), pred_poly)
       #print("shape",land_use_geoseries.shape)
       #print("++++++++++++")

       #print(land_use_geoseries.overlaps(pred_poly).any())
       if (forest_geoseries.overlaps(pred_poly).any()==True) or (forest_geoseries.contains(pred_poly).any()==True) or (forest_geoseries.within(pred_poly).any()==True):

           print(row_counter, "match", row[0])  
           predictions_in_forest_df = predictions_in_forest_df.append(row, ignore_index=True)
       else:
           print(row_counter, "    no match", row[0])  
       row_counter = row_counter + 1   

if not predictions_in_forest_df.empty:
    predictions_in_forest_df.to_file(predictions_in_forest_path)
    print("Total predictions in forested areas:", len(predictions_in_forest_df))
    print("of total raw predictions:", len(predictions_polys))
else:
    print("empty")

In [ ]:
geotif_fp = '/content/drive/MyDrive/crane_pennsylvania/lidar_files/slope/slope_0-597/slope_15001970pas10_utm.tif'
minx = 294
miny = 1478
maxx = 337
maxy = 1521

# pos 197092.296875 4381544.0 197092.296875 4381501.0 197135.296875 4381501.0 197135.296875 4381544.0


ds = gdal.Open(geotif_fp)
print(ds.GetProjection())
# open the dataset and get the geo transform matrix

xoffset, px_w, rot1, yoffset, rot2,px_h = ds.GetGeoTransform()

#print("xoffset, px_w, rot1, yoffset, px_h, rot2",xoffset, px_w, rot1, yoffset, px_h, rot2)
print("minx,miny,maxx,maxy",minx,miny,maxx,maxy)

pos1x,pos1y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,minx,miny)
pos2x,pos2y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,minx,maxy)
pos3x,pos3y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,maxx,maxy)
pos4x,pos4y = get_posx_posy(xoffset, px_w, rot1, yoffset, px_h, rot2,maxx,miny)
coords = [(pos1x,pos1y), (pos2x,pos2y), (pos3x,pos3y), (pos4x,pos4y)]

print("pos",pos1x,pos1y,pos2x,pos2y,pos3x,pos3y,pos4x,pos4y)
poly = Polygon(coords)

# Convert Polygons to Points and remove Duplicates
For each area, load the polygons.
Check if any existing points from previous areas processed are inside any of the polygons of this area.
If there are "matches" (duplicates), the polygons are removed (and stored in a dataframe of duplicates)
The left over unique polygons are then processed for their centroids.
These points are stored for output and also used to process the polygons for the remaining areas so see if there are duplicates for in the polygons of the remaining areas.

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import csv
def preds_to_points(area, pred_poly_folder, all_pred_points_df, dup_pred_polys_df, area_crs = 26918):
    # print(all_pred_points_df.shape)
    print(area)
    area_pred_polys_path = os.path.join(pred_poly_folder,(area + "_predictions_in_forest.shp"))
    print(area_pred_polys_path)
    if os.path.exists(area_pred_polys_path):
        area_pred_polys = gpd.read_file(area_pred_polys_path)
        # selection = pred_data[0:]
        # print(list(area_pred_polys))
        # print(area_pred_polys.shape)

        matched_pred_polys = list()    
        area_pred_polys.to_crs(area_crs)
        # for pred_poly in area_pred_polys:
        for index, row in area_pred_polys.iterrows():
            #print("row",row[0],row[1],row[2])
            pred_poly = row[2]
            # any_points = all_pred_points_df.within(pred_poly.loc[0, 'geometry'])
            any_points = all_pred_points_df.within(pred_poly)
            #print("any_points",any_points)
            if(any(any_points) == True):
                # print("MATCHES")
                matched_pred_polys.append(str(row[0]))
                
        # print(matched_pred_polys)
        # If there is more than 0 matches, remove them from the dataframe
        if(len(matched_pred_polys) > 0 ):
            print("area_pred_polys len before ",len(area_pred_polys))
            for mpp in matched_pred_polys:
                index_matches = area_pred_polys[area_pred_polys['id'] == mpp].index
                #dup_row = area_pred_polys.loc([area_pred_polys['id'] == mpp]
                # get the duplicate row
                dup_row = area_pred_polys.loc[area_pred_polys['id'] == mpp]
                #print("dup_row....",dup_row,dup_row['id'])
                #print("index_matches",index_matches)
                #print("dup_pred_polys_df len before",len(dup_pred_polys_df))
                # put the duplicate row into a dataframe it can be saved to check it.
                dup_pred_polys_df = dup_pred_polys_df.append(dup_row, ignore_index=True)
                #print("dup_pred_polys_df len after",len(dup_pred_polys_df))
                area_pred_polys.drop(index_matches, inplace = True)
            # area_pred_polys.drop(matched_pred_polys)
            print("area_pred_polys len after ",len(area_pred_polys))
            
        # Create an empty geopandas GeoDataFrame
        area_pred_points_df = gpd.GeoDataFrame()
        #area_pred_points_df.crs = {'init':'epsg:' + str(area_crs)}
        area_pred_points_df.crs = ('EPSG:' + str(area_crs))
        
        area_pred_points_df['geometry'] = area_pred_polys.centroid
        # make an id
        id_list = np.arange(1,len(area_pred_polys.centroid)+1)
        # print(id_list)
        id_list = [(area + "-" + (("000"+str(i))[-4:])) for i in id_list]
        # print(id_list)
        area_pred_points_df['id'] = id_list
        area_pred_points_df['score'] = area_pred_polys['score']
        
        dataframesList = [all_pred_points_df, area_pred_points_df]
        all_pred_points_df = gpd.GeoDataFrame(pd.concat(dataframesList, ignore_index=True), crs=dataframesList[0].crs)
        
        print("Total points area:", len(all_pred_points_df)," Total duplicates:", len(dup_pred_polys_df))
        return all_pred_points_df, dup_pred_polys_df, area_pred_points_df
    else:
        print("Does not exist.",area_pred_polys_path," points area:", len(all_pred_points_df)," Total duplicates:", len(dup_pred_polys_df))
        area_pred_points_df = gpd.GeoDataFrame()
        return all_pred_points_df, dup_pred_polys_df, area_pred_points_df


construction_type = "charcoal_hearth_hill"
cfg_name = 'cfg20200826T2315'
area_crs = 26918
area_name = batch_group

# Create an empty geopandas GeoDataFrame
all_pred_points_df = gpd.GeoDataFrame()
all_pred_points_df['geometry'] = None
all_pred_points_df['id'] = None
all_pred_points_df.crs = ('EPSG:'+str(area_crs))

# Create an empty geopandas GeoDataFrame for duplicates
dup_pred_polys_df = gpd.GeoDataFrame()
dup_pred_polys_df['geometry'] = None
dup_pred_polys_df['id'] = None
dup_pred_polys_df.crs = ('EPSG:'+str(area_crs))

poly_folder = prediction_fp
all_points_outfp = os.path.join(poly_folder, (area_name+"_all_hearth_prediction_points_in_forest.shp"))
dup_polys_outfp = os.path.join(poly_folder, (area_name+"_duplicate_hearth_prediction_polys_in_forest.shp"))

all_pred_points_df, dup_pred_polys_df, area_pred_points_df = preds_to_points(area_name, poly_folder, all_pred_points_df, dup_pred_polys_df, area_crs)
area_points_outfp = os.path.join(poly_folder, (area_name+"_hearth_prediction_points_in_forest.shp"))
# Write the data into that Shapefile
if not area_pred_points_df.empty:    
    area_pred_points_df.to_file(area_points_outfp)
    print("Total points for area:", len(area_pred_points_df))

# Write the data into that Shapefile
if not all_pred_points_df.empty:    
    all_pred_points_df.to_file(all_points_outfp)
    print("Total points:", len(all_pred_points_df))
if not dup_pred_polys_df.empty:
    dup_pred_polys_df.to_file(dup_polys_outfp)
    print("Total duplicate polys:", len(dup_pred_polys_df))

In [ ]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 1.9 MB/s 


In [ ]:
# Make a shapefile outlining the dem tif boundaries
import rasterio
import pandas as pd
import geopandas as gpd

import os
from os import listdir
df = ""
counter = 0
for dem_filename in listdir(dem_tif_fp):
    print(dem_filename)
    ra = rasterio.open(os.path.join(dem_tif_fp,dem_filename))
    bounds  = ra.bounds
    from shapely.geometry import box
    geom = box(*bounds)
    tdf = gpd.GeoDataFrame({"id":counter,"filename":dem_filename,"geometry":[geom]})
    if len(df) == 0:
        df = tdf
    else:
        df = gpd.GeoDataFrame( pd.concat( [df,tdf], ignore_index=True) )
        counter = counter + 1
df.to_file(os.path.join(prediction_fp,"dem_boundary.shp"))
print("the file is in ",prediction_fp)

USGS_one_meter_x26y449_PA_South_Central_B1_2017.tif
USGS_one_meter_x26y450_PA_South_Central_B1_2017.tif
USGS_one_meter_x26y451_PA_South_Central_B1_2017.tif


RasterioIOError: ignored

In [ ]:
import json
import os
with open("/content/drive/MyDrive/crane_pennsylvania/pa_michaux_forest_area_rch/all_pa.json", 'r') as dem_data_file:
    dem_data = json.load(dem_data_file)
    
    #print(json.dumps(dem_data, indent=4, sort_keys=True))
    print("There are ",len(dem_data['items'])," files.")
    file_counter = 0
    for item in dem_data['items']:
        title = item['title'] 
        print(title)

"""        
        dem_file_name = (title.strip()).replace(' ', "_")+ ".tif"
        dem_file_path = os.path.join(dem_tif_fp,dem_file_name)

#        url_of_dem = url_of_dem_folder + dem_file_name 
#        print(file_counter, url_of_dem)
#        file_counter = file_counter + 1
        try:
            if not os.path.exists(dem_file_path):
                downloadObj.downloadFile(url_of_dem,dem_tif_fp)
                time.sleep(10)
            else:
                print(dem_file_path, " exists.")
        except:
            print("exception:",url_of_dem,dem_tif_fp)
"""

There are  480  files.
USGS one meter x23y441 PA South Central B1 2017
USGS one meter x24y441 PA South Central B1 2017
USGS one meter x24y442 PA South Central B1 2017
USGS one meter x24y443 PA South Central B1 2017
USGS one meter x25y441 PA South Central B1 2017
USGS one meter x25y442 PA South Central B1 2017
USGS one meter x25y443 PA South Central B1 2017
USGS one meter x25y444 PA South Central B1 2017
USGS one meter x25y445 PA South Central B1 2017
USGS one meter x25y448 PA South Central B1 2017
USGS one meter x25y449 PA South Central B1 2017
USGS one meter x25y450 PA South Central B1 2017
USGS one meter x26y441 PA South Central B1 2017
USGS one meter x26y442 PA South Central B1 2017
USGS one meter x26y443 PA South Central B1 2017
USGS one meter x26y444 PA South Central B1 2017
USGS one meter x26y445 PA South Central B1 2017
USGS one meter x26y446 PA South Central B1 2017
USGS one meter x26y447 PA South Central B1 2017
USGS one meter x26y448 PA South Central B1 2017
USGS one meter x2

'        \n        dem_file_name = (title.strip()).replace(\' \', "_")+ ".tif"\n        dem_file_path = os.path.join(dem_tif_fp,dem_file_name)\n\n#        url_of_dem = url_of_dem_folder + dem_file_name \n#        print(file_counter, url_of_dem)\n#        file_counter = file_counter + 1\n        try:\n            if not os.path.exists(dem_file_path):\n                downloadObj.downloadFile(url_of_dem,dem_tif_fp)\n                time.sleep(10)\n            else:\n                print(dem_file_path, " exists.")\n        except:\n            print("exception:",url_of_dem,dem_tif_fp)\n'

In [ ]:
# This may blow up Google Colab - I think the 10k X 10k JPG is too large

import os
from os import listdir
import cv2
import skimage
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from google.colab.patches import cv2_imshow
from matplotlib import pyplot
from matplotlib.pyplot import figure

training_annot_file_path = ""

font = cv2.FONT_HERSHEY_SIMPLEX 
font_scale = 1
font_color = (0, 200, 0) 
font_thickness = 2

file_counter = 0

for prediction_image in listdir(slope_tif_tiles_jpgs_fp):
    print(file_counter, prediction_image)
    if(file_counter>=0 and file_counter<5000):
        annotation_file = prediction_image[:-4]+".xml"
        image = cv2.imread(os.path.join(slope_tif_tiles_jpgs_fp,prediction_image))
        print(image.shape)
        prediction_annot_file_path = os.path.join(prediction_xmls_fp,annotation_file)

        overlay = image.copy()
        output = image.copy()
        #cv2_imshow(output)    
        font_color = (255, 0, 255)
        font_scale = .6
        font_thickness = 1

        print(prediction_annot_file_path)
        if(os.path.exists(prediction_annot_file_path)):
            tree_train = ElementTree.parse(prediction_annot_file_path)
            root_train = tree_train.getroot()
            for obj in root_train.findall('./object'):
                box = obj.find('bndbox')
                xmin = int(box.find('xmin').text)
                ymin = int(box.find('ymin').text)
                xmax = int(box.find('xmax').text)
                ymax = int(box.find('ymax').text)
                box_num = str(obj.find('number').text)
                score = float(obj.find('score').text)
                if(ymin>ymax):
                    ytemp = ymin
                    ymin = ymax
                    ymax=ytemp
                if(xmin>xmax):
                    xtemp = xmin
                    xmin = xmax
                    xmax=xtemp            
                coors = [xmin, ymin, xmax, ymax]
                crop_output = output[ymin:ymax, xmin:xmax]
        
                cv2.rectangle(output, (xmin, ymin), (xmax, ymax),(255, 0, 255), 2)
                cv2.putText(output, f'{score:.3f}' , (xmin+0, ymin-17), font, font_scale, font_color, font_thickness, cv2.LINE_AA)
            print(root_train.findall('./object'))
            objs = root_train.findall('./object')
            if(len(objs)>0):
                cv2_imshow(output)
                file_counter = file_counter +1
                if file_counter > 20:
                    break
        else:
            print("No prediction annotation file for this image.  This is likely ok.")



KeyboardInterrupt: ignored

In [ ]:
# one time move
import os
import shutil
import sys
#shutil.move()
b2 = '/content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b2_2017/backup/'
pa_south_central_b1_xmls_fp =  '/content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b1_2017/xmls/'
if os.path.exists(b2):
    print("good")
else:
    print(b2, "not there")
    sys.exit("not there")
if os.path.exists(pa_south_central_b1_xmls_fp):
    print("good")
else:
    print(pa_south_central_b1_xmls_fp, "not there")
    sys.exit("not there")
file_counter = 0
move_file_counter = 0
for file_name in os.listdir(pa_south_central_b1_xmls_fp):
    origin_file_path = os.path.join(pa_south_central_b1_xmls_fp,file_name)
    
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".xml"):
            if(file_name[23:47]=="PA_South_Central_B2_2017"):
                destination_file_path = os.path.join(b2,file_name)  
                shutil.move(origin_file_path, destination_file_path)
                print(destination_file_path)
                move_file_counter = move_file_counter + 1
        file_counter = file_counter + 1
print(file_counter, move_file_counter)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b2_2017/backup/USGS_one_meter_x33y458_PA_South_Central_B2_2017r09c01.xml
/content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b2_2017/backup/USGS_one_meter_x33y458_PA_South_Central_B2_2017r09c02.xml
/content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b2_2017/backup/USGS_one_meter_x33y458_PA_South_Central_B2_2017r09c03.xml
/content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b2_2017/backup/USGS_one_meter_x33y458_PA_South_Central_B2_2017r09c04.xml
/content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b2_2017/backup/USGS_one_meter_x33y458_PA_South_Central_B2_2017r09c05.xml
/content/drive/MyDrive/crane_pennsylvania/predictions/project_pa_south_central_b2_2017/backup/USGS_one_meter_x33y458_PA_South_Central_B2_2017r09c06.xml
/content/drive/MyDrive/crane_pennsylv